<a href="https://colab.research.google.com/github/islamhanii/HOG/blob/main/CV_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import numpy as np
from keras.datasets import mnist
from sklearn import svm
from sklearn.svm import SVC
from sklearn.metrics import classification_report,accuracy_score

In [ ]:
def getFeatures(data):
  feature_vectors = []
  for i in range(len(data)):
    image = data[i]
    image = np.pad(image, ((0,4),(0,4)), constant_values=255)

    gradient_kernel_x = np.array([[-1, 0, 1]])
    gradient_kernel_y = np.reshape(gradient_kernel_x, (3,1))

    gradient_x = cv2.filter2D(image,-1,gradient_kernel_x)
    gradient_y = cv2.filter2D(image,-1,gradient_kernel_y)

    gradient_x_square = np.power(gradient_x, 2)
    gradient_y_square = np.power(gradient_y, 2)

    sum_squares = gradient_x_square + gradient_y_square
    grad_magnitude = np.sqrt(sum_squares)

    grad_direction = np.arctan(gradient_y/(gradient_x+0.00000001))
    grad_direction = np.rad2deg(grad_direction)
    grad_direction = grad_direction%180

    feature_vector = []

    for i in range(0,len(image),8):
        for j in range(0,len(image[0]),8):
            block_magn = grad_magnitude[i:i+16,j:j+16]
            block_seta = grad_direction[i:i+16,j:j+16]
            
            sum = 0
            block_vector = []
            for x in range(0,len(block_magn),8):
                for y in range(0,len(block_magn[0]),8):
                    cell_magn = block_magn[x:x+8,y:y+8]
                    cell_seta = block_seta[x:x+8,y:y+8]
                    
                    histogram = {0:0, 20:0, 40:0, 60:0, 80:0, 100:0, 120:0, 140:0, 160:0}
                    for n in range(len(cell_magn)):
                        for m in range(len(cell_magn[0])):
                            seta = cell_seta[n][m]
                            value = cell_magn[n][m]
                            sum += cell_magn[n][m]
                    
                            for s in histogram.keys():
                                div = (seta-s)/20
                                if(seta == 0):
                                    histogram[s] += value
                                    break
                                
                                elif(div<=1):
                                    histogram[s] += value*(1-div)
                                    if(s==160):
                                        histogram[0] += value*div
                                        break
                                    histogram[s+20] += value*div
                                    break
                    
                    histogram = list(histogram.values())
                    block_vector.append(histogram)
            
            for x in range(len(block_vector)):
                for y in range(len(block_vector[0])):
                    if(sum!=0):
                        block_vector[x][y] /= sum
                
            feature_vector += block_vector
            
    feature_vectors.append(feature_vector)
  return feature_vectors

In [ ]:
(train_X, train_y), (test_X, test_y) = mnist.load_data()

train_imgs = np.array(~train_X[:50000] , dtype=float)
train_labels = np.array(train_y[:50000])
test_imgs = np.array(~test_X[:10000] , dtype=float)
test_labels = np.array(test_y[:10000])

train_features = np.array(getFeatures(train_imgs))
test_features = np.array(getFeatures(test_imgs))

nsamples, nx, ny = train_features.shape
train_features = train_features.reshape((nsamples,nx*ny))

nsamples, nx, ny = test_features.shape
test_features = test_features.reshape((nsamples,nx*ny))

clf = svm.SVC()
clf.fit(train_features, train_labels)
pred=clf.predict(test_features)

print("Accurate percentage"+str(accuracy_score(test_labels,pred)))
print("\n")
print(classification_report(test_labels,pred))

Accurate percentage0.9723


              precision    recall  f1-score   support

           0       0.97      0.99      0.98       980
           1       0.99      0.99      0.99      1135
           2       0.98      0.98      0.98      1032
           3       0.96      0.97      0.96      1010
           4       0.97      0.97      0.97       982
           5       0.98      0.98      0.98       892
           6       0.99      0.98      0.98       958
           7       0.97      0.95      0.96      1028
           8       0.97      0.97      0.97       974
           9       0.96      0.95      0.95      1009

    accuracy                           0.97     10000
   macro avg       0.97      0.97      0.97     10000
weighted avg       0.97      0.97      0.97     10000

